In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./src/final_dataset.csv')
df.columns

Index(['date', 'season', 'league_name', 'team_long_name_home',
       'team_short_name_home', 'team_short_name_away', 'team_long_name_away',
       'home_team_goal', 'away_team_goal', 'match_result',
       'home_players_average_rating', 'away_players_average_rating',
       'home_players_average_score', 'away_players_average_score',
       'away_team_score', 'home_team_score', 'home_team_red_cards',
       'away_team_red_cards', 'avg_bet_home', 'avg_bet_draw', 'avg_bet_away'],
      dtype='object')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14916 entries, 0 to 14915
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         14916 non-null  object 
 1   season                       14916 non-null  object 
 2   league_name                  14916 non-null  object 
 3   team_long_name_home          14916 non-null  object 
 4   team_short_name_home         14916 non-null  object 
 5   team_short_name_away         14916 non-null  object 
 6   team_long_name_away          14916 non-null  object 
 7   home_team_goal               14916 non-null  int64  
 8   away_team_goal               14916 non-null  int64  
 9   match_result                 14916 non-null  int64  
 10  home_players_average_rating  14916 non-null  float64
 11  away_players_average_rating  14916 non-null  float64
 12  home_players_average_score   14916 non-null  float64
 13  away_players_ave

In [4]:
df1 = df[['league_name', 'team_long_name_home',
       'team_short_name_home',]]
df1.columns = ['league_name', 'team_long_name',
       'team_short_name']
df2 = df[['league_name', 'team_long_name_away',
       'team_short_name_away',]]
df2.columns = ['league_name', 'team_long_name',
       'team_short_name']
concat_df = pd.concat([df1, df2], ignore_index=True)
concat_df.drop_duplicates(inplace=True)
cross_joined_df = pd.concat(
    [
        sub_df.merge(sub_df, on='league_name').query('team_long_name_x != team_long_name_y')
        for _, sub_df in concat_df.groupby('league_name')
    ]
)
cross_joined_df.columns = ['league_name', 'team_long_name_home',
       'team_short_name_home','team_long_name_away',
       'team_short_name_away']
cross_joined_df

,league_name,team_long_name_home,team_short_name_home,team_long_name_away,team_short_name_away
1,Belgium Jupiler League,RSC Anderlecht,AND,KV Mechelen,MEC
2,Belgium Jupiler League,RSC Anderlecht,AND,Standard de Liège,STL
3,Belgium Jupiler League,RSC Anderlecht,AND,Beerschot AC,BAC
4,Belgium Jupiler League,RSC Anderlecht,AND,KRC Genk,GEN
5,Belgium Jupiler League,RSC Anderlecht,AND,SV Zulte-Waregem,ZUL
...,...,...,...,...,...
955,Spain LIGA BBVA,UD Las Palmas,LAS,Granada CF,GRA
956,Spain LIGA BBVA,UD Las Palmas,LAS,RC Celta de Vigo,CEL
957,Spain LIGA BBVA,UD Las Palmas,LAS,Elche CF,ELC
958,Spain LIGA BBVA,UD Las Palmas,LAS,SD Eibar,EIB


In [5]:
temp_df=df
temp_df['date'] = pd.to_datetime(temp_df['date'])
temp_df=temp_df.sort_values('date').drop_duplicates(['league_name', 'team_long_name_home',
       'team_short_name_home', 'team_short_name_away', 'team_long_name_away',], keep='last')
cross_joined_df=pd.merge(cross_joined_df, temp_df, on=['league_name', 'team_long_name_home',
       'team_short_name_home','team_long_name_away',
       'team_short_name_away'], how='left')
cross_joined_df[['date', 'season', 'league_name', 'team_long_name_home',
       'team_short_name_home', 'team_short_name_away', 'team_long_name_away',
       'home_team_goal', 'away_team_goal', 'match_result',
       'home_players_average_rating', 'away_players_average_rating',
       'home_players_average_score', 'away_players_average_score',
       'away_team_score', 'home_team_score', 'home_team_red_cards',
       'away_team_red_cards', 'avg_bet_home', 'avg_bet_draw', 'avg_bet_away']].to_csv(
    './src/new_variables.csv', index=False)

,date,season,league_name,team_long_name_home,team_short_name_home,team_short_name_away,team_long_name_away,home_team_goal,away_team_goal,match_result,...,away_players_average_rating,home_players_average_score,away_players_average_score,away_team_score,home_team_score,home_team_red_cards,away_team_red_cards,avg_bet_home,avg_bet_draw,avg_bet_away
0,2015-10-04,2015/2016,Belgium Jupiler League,RSC Anderlecht,AND,MEC,KV Mechelen,1.0,1.0,0.0,...,66.64,66.72,59.82,48.38,53.38,0.0,0.0,0.699301,0.225225,0.140647
1,2014-10-26,2014/2015,Belgium Jupiler League,RSC Anderlecht,AND,STL,Standard de Liège,0.0,2.0,-1.0,...,69.18,64.57,63.46,55.00,54.75,0.0,0.0,0.613497,0.261780,0.201613
2,2012-08-04,2012/2013,Belgium Jupiler League,RSC Anderlecht,AND,BAC,Beerschot AC,1.0,0.0,1.0,...,65.82,65.68,60.42,49.12,48.75,0.0,0.0,0.800000,0.187266,0.095694
3,2015-09-13,2015/2016,Belgium Jupiler League,RSC Anderlecht,AND,GEN,KRC Genk,0.0,0.0,0.0,...,67.45,66.64,60.90,51.75,53.38,0.0,0.0,0.606061,0.257732,0.196850
4,2015-02-01,2014/2015,Belgium Jupiler League,RSC Anderlecht,AND,ZUL,SV Zulte-Waregem,0.0,0.0,0.0,...,65.50,62.92,59.52,46.88,53.38,0.0,0.0,0.704225,0.223214,0.139082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6527,2015-12-30,2015/2016,Spain LIGA BBVA,UD Las Palmas,LAS,GRA,Granada CF,4.0,1.0,1.0,...,70.91,61.94,63.97,55.75,48.62,0.0,0.0,0.478469,0.306748,0.267380
6528,NaT,NaN,Spain LIGA BBVA,UD Las Palmas,LAS,CEL,RC Celta de Vigo,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6529,NaT,NaN,Spain LIGA BBVA,UD Las Palmas,LAS,ELC,Elche CF,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6530,NaT,NaN,Spain LIGA BBVA,UD Las Palmas,LAS,EIB,SD Eibar,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
